In [1]:
from square.client import Client
import json, uuid, datetime
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine import URL

import time

tic = time.time()

In [2]:
root_path = ''

with open(root_path+'appdata/square_auth.json') as f:
    temp = json.load(f)
    access_token = temp['access_token']
    environment = temp['environment'] 

In [3]:
client = Client(access_token=access_token, environment=environment)

In [4]:
body = {
    "product_types": [
        "REGULAR"
    ]
}

result = client.catalog.search_catalog_items(body=body)

In [5]:
# for body in result.body['items']:
#     print(body)
#     print('-'*100)

In [6]:
item_list = pd.DataFrame(columns=['object_id', 'upc'])

for body in result.body['items']:
    for data in body['item_data']['variations']:
        try:
            item_list.loc[len(item_list)]= {'object_id': data['id'], 'upc': data['item_variation_data']['upc']}
        except:
            print(data['id'])

UGNPVAAYCZZAXCYT5U42F2RM
5JK27VJV4I4X2RGCD6CVJANI
CMNF7457MTFJCNM5Q2MZB37J
OLFO7VF2ZQXZYXK76TLRTIOE
OR37JNIWLUTPT2OHPTIEPIZK


In [7]:
item_list

,object_id,upc
0,WM2M7LQ7NNQ4W5UWJDNLTRFL,650270541019
1,ER5A6FQTK6XOVPNRESQ5N2T7,650270541026
2,YHSYCGWO474CZ7CBWFMUET2J,650270541033
3,URXHG7SHVRHOCGOS76MVAKZO,650270541040
4,DYDLQXE7AAL7FNAPIOM3O7JG,650270541057
...,...,...
1057,6TZGICRHB7T6SSGA73U7RMVX,827298861935
1058,AXJU4KJN5R2JVQBGRTNGUM4Y,827298861942
1059,54PLP7W6Z7JY4XZWC45MS22E,827298861959
1060,T5MH7V5EHV26VBL2I5E4L55J,827298861966


In [8]:
with open(root_path+'appdata/db_auth.json') as f:
    temp = json.load(f)
    server = temp['server']
    database = temp['database'] 
    username = temp['username'] 
    password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

query = "SELECT ItemLookupCode, Quantity\
        FROM dbo.Item Item \
        WHERE Item.DepartmentID IN (2, 4, 6) AND Item.Inactive = 0\
        ORDER BY ItemLookupCode;"

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'Quantity':'int64'})

fromPOS.fillna('', inplace=True)

fromPOS.columns=['Item Lookup Code', 'Qty On Hand']

In [9]:
test = item_list.merge(fromPOS[['Item Lookup Code', 'Qty On Hand']], how='left', left_on='upc', right_on='Item Lookup Code')

In [10]:
test.loc[test['Qty On Hand']<0, 'Qty On Hand'] = 0
test.drop('Item Lookup Code', axis=1, inplace=True)
test.dropna(ignore_index=True ,inplace=True)

In [11]:
batch_size = 100  # 100 is maximum number for batch update supported by Square API

for i in range(0, len(test), batch_size):
    batch = range(i, min(i+batch_size, len(test)))
    body = {
        "idempotency_key": str(uuid.uuid1()),
        "changes": []
    }
    for index in batch:
        try:
            body['changes'].append(
                {
                    "type": "PHYSICAL_COUNT",
                    "physical_count": {
                        "catalog_object_id": test['object_id'][index],
                        "state": "IN_STOCK",
                        "location_id": "7BRJFTZVH3FSX",
                        "quantity": str(test['Qty On Hand'][index]),
                        "occurred_at": datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H:%M:%S.000Z')
                    }
                }
            )
        except:
            print(test.loc[index])
    result = client.inventory.batch_change_inventory(body=body)

    if result.is_success():
        print(result.body)
    elif result.is_error():
        print(result.errors)


{}
{}
{'counts': [{'catalog_object_id': 'EQSEKTN5VJKDIDTIO3NCUQMC', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': '7BRJFTZVH3FSX', 'quantity': '0', 'calculated_at': '2024-01-30T22:48:01.963Z'}, {'catalog_object_id': 'U66ZSBI6JOPJJJTE67LVE7C7', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': '7BRJFTZVH3FSX', 'quantity': '1', 'calculated_at': '2024-01-30T22:48:01.964Z'}]}
{'counts': [{'catalog_object_id': '4UFP2DYCODMRIFZDPZOJVEIE', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': '7BRJFTZVH3FSX', 'quantity': '1', 'calculated_at': '2024-01-30T22:48:02.54Z'}]}
{}
{}
{}
{}
{}
{}
{}


In [12]:
toc = time.time()

print(toc - tic)

-9.281332015991211
